In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set(style="white", color_codes=True)
sns.set(font_scale=1)

/Users/xinyutian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/xinyutian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# load data
train_data = pd.read_csv('../raw_data/train_data.csv')
name_df = pd.read_csv('../data_preprocess/df/texts/name_df.csv')
description_df = pd.read_csv('../data_preprocess/df/texts/description_df.csv')
host_verifications_df = pd.read_csv('../data_preprocess/df/texts/host_verifications_df.csv')
neighbourhood_cleansed_df = pd.read_csv('../data_preprocess/df/texts/neighbourhood_cleansed_df.csv')
neighbourhood_group_cleansed_df = pd.read_csv('../data_preprocess/df/texts/neighbourhood_group_cleansed_df.csv')
property_type_df = pd.read_csv('../data_preprocess/df/texts/property_type_df.csv')
room_type_df = pd.read_csv('../data_preprocess/df/texts/room_type_df.csv')
bathrooms_text_df = pd.read_csv('../data_preprocess/df/texts/bathrooms_text_df.csv')
amenities_df = pd.read_csv('../data_preprocess/df/texts/amenities_df.csv')


In [41]:
test_data = pd.read_csv('../raw_data/test_data.csv')
name_df_test = pd.read_csv('../data_preprocess/df/texts/name_df_test.csv')
description_df_test = pd.read_csv('../data_preprocess/df/texts/description_df_test.csv')
host_verifications_df_test = pd.read_csv('../data_preprocess/df/texts/host_verifications_df_test.csv')
neighbourhood_cleansed_df_test = pd.read_csv('../data_preprocess/df/texts/neighbourhood_cleansed_df_test.csv')
neighbourhood_group_cleansed_df_test = pd.read_csv('../data_preprocess/df/texts/neighbourhood_group_cleansed_df_test.csv')
property_type_df_test = pd.read_csv('../data_preprocess/df/texts/property_type_df_test.csv')
room_type_df_test = pd.read_csv('../data_preprocess/df/texts/room_type_df_test.csv')
bathrooms_text_df_test = pd.read_csv('../data_preprocess/df/texts/bathrooms_text_df_test.csv')
amenities_df_test = pd.read_csv('../data_preprocess/df/texts/amenities_df_test.csv')

In [4]:
price_data = pd.read_csv('../raw_data/train.csv')
price_df= price_data[['price']]

In [5]:
data = pd.concat([train_data, name_df, description_df, host_verifications_df, neighbourhood_cleansed_df, neighbourhood_group_cleansed_df, property_type_df, room_type_df, bathrooms_text_df, amenities_df, price_df], axis=1)

In [42]:
test_data_all = pd.concat([test_data, name_df_test, description_df_test, host_verifications_df_test, neighbourhood_cleansed_df_test, neighbourhood_group_cleansed_df_test, property_type_df_test, room_type_df_test, bathrooms_text_df_test, amenities_df_test], axis=1)

In [6]:
data.head()

,host_id,host_since,host_identity_verified,accommodates,price,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,...,zehr,zero,zeros,zest,zline,zogics,zologic,zone,zoologics,price
0,205539927,2018-07-27,1,3,0.0,30,30,30,30.0,28,...,0,0,0,0,0,0,0,0,0,0.0
1,7599550,2013-07-19,1,6,3.0,3,3,3,3.0,13,...,0,0,0,0,0,0,0,0,0,3.0
2,271118401,2019-06-24,1,8,2.0,3,3,3,3.0,19,...,0,0,0,0,0,0,0,0,0,2.0
3,429951162,2021-11-01,1,7,3.0,28,28,28,28.0,30,...,0,0,0,0,0,0,0,0,0,3.0
4,501999278,2023-02-20,1,2,4.0,1,1,1,1.0,12,...,0,0,0,0,0,0,0,0,0,4.0


In [43]:
test_data_all.head()

,host_id,host_since,host_identity_verified,accommodates,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,availability_60,...,xeba,yamaha,year,years,yoga,you,your,zenith,zero,zline
0,27551941,2015-02-10,1,6,30,30,30,30.0,0,18,...,0,0,0,0,0,0,0,0,0,0
1,483914016,2022-10-17,1,8,2,2,2,2.0,15,37,...,0,0,0,0,0,0,0,0,0,0
2,491587138,2022-12-15,1,8,1,1,1,1.0,15,38,...,0,0,0,0,0,0,0,0,0,0
3,63344837,2016-03-17,1,8,2,2,2,2.0,26,47,...,0,0,0,0,0,0,0,0,0,0
4,38487199,2015-07-14,1,8,30,30,30,30.0,10,20,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.dtypes

host_id                     int64
host_since                 object
host_identity_verified      int64
accommodates                int64
price                     float64
                           ...   
zogics                      int64
zologic                     int64
zone                        int64
zoologics                   int64
price                     float64
Length: 31010, dtype: object

In [44]:
test_data_all.dtypes

host_id                    int64
host_since                object
host_identity_verified     int64
accommodates               int64
minimum_nights             int64
                           ...  
you                        int64
your                       int64
zenith                     int64
zero                       int64
zline                      int64
Length: 20345, dtype: object

In [8]:
data.drop(['host_since'], axis=1, inplace=True)

In [45]:
test_data_all.drop(['host_since'], axis=1, inplace=True)

In [9]:
data_train = data.loc[:,~data.columns.duplicated()]

In [46]:
test_data_all = test_data_all.loc[:,~test_data_all.columns.duplicated()]

In [10]:
data_train.shape

(15441, 29311)

In [47]:
test_data_all.shape

(6291, 19105)

In [11]:
data_train.head()

,host_id,host_identity_verified,accommodates,price,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,availability_60,...,yardley,yardleys,yardly,zehr,zeros,zest,zline,zogics,zologic,zoologics
0,205539927,1,3,0.0,30,30,30,30.0,28,58,...,0,0,0,0,0,0,0,0,0,0
1,7599550,1,6,3.0,3,3,3,3.0,13,22,...,0,0,0,0,0,0,0,0,0,0
2,271118401,1,8,2.0,3,3,3,3.0,19,37,...,0,0,0,0,0,0,0,0,0,0
3,429951162,1,7,3.0,28,28,28,28.0,30,60,...,0,0,0,0,0,0,0,0,0,0
4,501999278,1,2,4.0,1,1,1,1.0,12,40,...,0,0,0,0,0,0,0,0,0,0


In [48]:
test_data_all.head()

,host_id,host_identity_verified,accommodates,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,availability_60,availability_90,...,wbluetooth,whirl,whish,whrilpool,wii,wof,women,xeba,yamaha,zline
0,27551941,1,6,30,30,30,30.0,0,18,48,...,0,0,0,0,0,0,0,0,0,0
1,483914016,1,8,2,2,2,2.0,15,37,61,...,0,0,0,0,0,0,0,0,0,0
2,491587138,1,8,1,1,1,1.0,15,38,65,...,0,0,0,0,0,0,0,0,0,0
3,63344837,1,8,2,2,2,2.0,26,47,77,...,0,0,0,0,0,0,0,0,0,0
4,38487199,1,8,30,30,30,30.0,10,20,37,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data_train['ac']

0        0
1        0
2        0
3        0
4        0
        ..
15436    0
15437    0
15438    0
15439    0
15440    0
Name: ac, Length: 15441, dtype: int64

In [49]:
test_data_all['ac']

0       0
1       0
2       0
3       0
4       0
       ..
6286    0
6287    0
6288    1
6289    0
6290    0
Name: ac, Length: 6291, dtype: int64

In [13]:
# save data_train
data_train.to_csv('data_train.csv', index=False)

In [50]:
test_data_all.to_csv('test_data_all.csv', index=False)

In [14]:
# split data
train, test = train_test_split(data_train, test_size=0.2, random_state=10)
train_y = train['price']
test_y = test['price']
train_x = train.drop(['price'], axis=1)
test_x = test.drop(['price'], axis=1)
train_x = train_x.drop(['id'], axis=1)
test_x = test_x.drop(['id'], axis=1)

In [15]:
train_y.dtypes

dtype('float64')

In [33]:
# xgboost
xgb_train = xgb.DMatrix(train_x, label=train_y, enable_categorical=True)
xgb_test = xgb.DMatrix(test_x, label=test_y, enable_categorical=True)

In [58]:
x_pred = test_data_all.drop(['id'], axis=1)

In [77]:
len(x_pred.columns)

19104

In [37]:
param_grid = {
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [100, 200, 300]
}

In [38]:
num_boost_round = 10000
nfold = 5
early_stopping_rounds = 10

for max_depth in param_grid['max_depth']:
    for learning_rate in param_grid['learning_rate']:
        params = {
            'max_depth': max_depth,
            'learning_rate': learning_rate,
            'objective': 'multi:softprob',
            'num_class': 6
        }
        print('Running Cross Validation, boosting round: {} of {}.'.format(num_boost_round, params))
        cv_results = xgb.cv(
            params,
            dtrain = xgb_train,
            num_boost_round=100,
            nfold=5,
            metrics={'merror'},
            # early_stopping_rounds=2,
            verbose_eval=True
        )

        # Store or print your results here
        print('hi')
        print(f"Results for params {params}: {cv_results['test-merror-mean'].min()}")

Running Cross Validation, boosting round: 10000 of {'max_depth': 5, 'learning_rate': 0.01, 'objective': 'multi:softprob', 'num_class': 6}.
[0]	train-merror:0.51144+0.00255	test-merror:0.55530+0.00748
[1]	train-merror:0.51156+0.00218	test-merror:0.55538+0.00810
[2]	train-merror:0.51142+0.00343	test-merror:0.55319+0.00665
[3]	train-merror:0.51154+0.00353	test-merror:0.55327+0.00657
[4]	train-merror:0.51083+0.00391	test-merror:0.55028+0.00677
[5]	train-merror:0.51008+0.00422	test-merror:0.55044+0.00628
[6]	train-merror:0.51046+0.00447	test-merror:0.55149+0.00578
[7]	train-merror:0.50976+0.00405	test-merror:0.54971+0.00620
[8]	train-merror:0.51117+0.00296	test-merror:0.55044+0.00800
[9]	train-merror:0.50919+0.00333	test-merror:0.54963+0.00716
[10]	train-merror:0.50808+0.00398	test-merror:0.54769+0.00987
[11]	train-merror:0.50708+0.00427	test-merror:0.54712+0.00956
[12]	train-merror:0.50629+0.00374	test-merror:0.54647+0.00849
[13]	train-merror:0.50583+0.00393	test-merror:0.54623+0.00900
[14

In [63]:
print(cv_results)

    train-merror-mean  train-merror-std  test-merror-mean  test-merror-std
0            0.241763          0.006477          0.541614         0.011814
1            0.202660          0.003939          0.534409         0.012209
2            0.184545          0.004015          0.529713         0.010294
3            0.172503          0.002481          0.523155         0.010594
4            0.161310          0.002099          0.519673         0.006942
..                ...               ...               ...              ...
95           0.017588          0.000902          0.476037         0.010176
96           0.017264          0.000934          0.475875         0.009706
97           0.017062          0.000829          0.475470         0.011714
98           0.016718          0.000829          0.475227         0.011127
99           0.016637          0.000870          0.474904         0.010740

[100 rows x 4 columns]


In [40]:
best_params = {'max_depth': 10, 'learning_rate': 0.05, 'objective': 'multi:softprob', 'num_class': 6}
best_model = xgb.train(best_params, xgb_train, num_boost_round=300, evals=[(xgb_test, 'test')])

[0]	test-mlogloss:1.75541
[1]	test-mlogloss:1.72309
[2]	test-mlogloss:1.69382
[3]	test-mlogloss:1.66666
[4]	test-mlogloss:1.64075
[5]	test-mlogloss:1.61742
[6]	test-mlogloss:1.59643
[7]	test-mlogloss:1.57644
[8]	test-mlogloss:1.55833
[9]	test-mlogloss:1.54055
[10]	test-mlogloss:1.52441
[11]	test-mlogloss:1.50812
[12]	test-mlogloss:1.49310
[13]	test-mlogloss:1.47858
[14]	test-mlogloss:1.46562
[15]	test-mlogloss:1.45249
[16]	test-mlogloss:1.44059
[17]	test-mlogloss:1.42941
[18]	test-mlogloss:1.41863
[19]	test-mlogloss:1.40862
[20]	test-mlogloss:1.39820
[21]	test-mlogloss:1.38917
[22]	test-mlogloss:1.37927
[23]	test-mlogloss:1.37121
[24]	test-mlogloss:1.36336
[25]	test-mlogloss:1.35504
[26]	test-mlogloss:1.34726
[27]	test-mlogloss:1.34035
[28]	test-mlogloss:1.33285
[29]	test-mlogloss:1.32668
[30]	test-mlogloss:1.32007
[31]	test-mlogloss:1.31355
[32]	test-mlogloss:1.30768
[33]	test-mlogloss:1.30222
[34]	test-mlogloss:1.29658
[35]	test-mlogloss:1.29131
[36]	test-mlogloss:1.28645
[37]	test-m

In [79]:
for col in train_x.columns:
    if col not in x_pred.columns:
        x_pred[col] = [0] * len(x_pred)

for col in x_pred.columns:
    if col not in train_x.columns:
        x_pred.drop([col], axis=1, inplace=True)

In [83]:
x_pred.columns

Index(['host_id', 'host_identity_verified', 'accommodates', 'minimum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90',
       ...
       'yajie', 'yardley', 'yardleys', 'yardly', 'zehr', 'zeros', 'zest',
       'zogics', 'zologic', 'zoologics'],
      dtype='object', length=29309)

In [88]:
x_pred

,host_id,host_identity_verified,accommodates,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,availability_60,availability_90,...,yajie,yardley,yardleys,yardly,zehr,zeros,zest,zogics,zologic,zoologics
0,27551941,1,6,30,30,30,30.0,0,18,48,...,0,0,0,0,0,0,0,0,0,0
1,483914016,1,8,2,2,2,2.0,15,37,61,...,0,0,0,0,0,0,0,0,0,0
2,491587138,1,8,1,1,1,1.0,15,38,65,...,0,0,0,0,0,0,0,0,0,0
3,63344837,1,8,2,2,2,2.0,26,47,77,...,0,0,0,0,0,0,0,0,0,0
4,38487199,1,8,30,30,30,30.0,10,20,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6286,417195791,1,10,31,31,31,31.0,30,60,90,...,0,0,0,0,0,0,0,0,0,0
6287,118262470,1,16,3,3,7,4.2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6288,518668175,1,6,1,1,2,1.1,5,17,37,...,0,0,0,0,0,0,0,0,0,0
6289,393586118,1,1,2,2,2,2.0,2,32,60,...,0,0,0,0,0,0,0,0,0,0


In [87]:
train_x.columns == x_pred.columns

array([ True,  True,  True, ...,  True,  True,  True])

In [90]:
cols_when_model_builds = best_model.feature_names

In [91]:
cols_when_model_builds

['host_id',
 'host_identity_verified',
 'accommodates',
 'minimum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_nights_avg_ntm',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'acton',
 'ada',
 'agoura',
 'agua',
 'aire',
 'alhambra',
 'altadena',
 'and',
 'angeles',
 'angles',
 'aparthotel',
 'apartment',
 'arcadia',
 'artesia',
 'avalon',
 'azusa',
 'balboa',
 'baldwin',
 'bar',
 'barn',
 'bath',
 'baths',
 'beach',
 'bed',
 'bedroom',
 'bedrooms',
 'beds',
 'bell',
 'bellflower',
 'beverly',
 'boat',
 'boutique',
 'breakfast',
 'bungalow',
 'burbank',
 'ca',
 'cabin',
 'calabasas',
 'california',
 'camino',
 'camperrv',
 'campsite',
 'canyon',
 'carson',
 'casa',
 'castaic',
 'castle',
 'century',
 'cerritos',
 'chalet',
 'chatsworth',
 'city',
 'claremont',
 'clarita',
 'commerce',
 'compton',
 'condo',
 'cottage',
 'country',
 'county',
 'covina',
 'crescentamontrose'

In [92]:
x_pred = x_pred[cols_when_model_builds]
xgb_pred = xgb.DMatrix(x_pred)

In [94]:
# predict
preds = best_model.predict(xgb_pred)

In [95]:
# save to csv
preds_df = pd.DataFrame(preds)
preds_df.to_csv('preds.csv', index=False)
# columns = ['id', 'price']


In [96]:
submission = pd.read_csv('preds.csv')
submission.head()

,0,1,2,3,4,5
0,0.026642,0.193753,0.256311,0.447479,0.067691,0.008125
1,0.019580,0.278200,0.430114,0.223611,0.046119,0.002377
2,0.009258,0.417058,0.421079,0.136868,0.014650,0.001086
3,0.002768,0.010895,0.089062,0.155363,0.690996,0.050916
4,0.008879,0.331591,0.207936,0.424389,0.026514,0.000691


In [99]:
submission['id'] = submission.index

In [100]:
submission.head()

,0,1,2,3,4,5,id
0,0.026642,0.193753,0.256311,0.447479,0.067691,0.008125,0
1,0.019580,0.278200,0.430114,0.223611,0.046119,0.002377,1
2,0.009258,0.417058,0.421079,0.136868,0.014650,0.001086,2
3,0.002768,0.010895,0.089062,0.155363,0.690996,0.050916,3
4,0.008879,0.331591,0.207936,0.424389,0.026514,0.000691,4


In [116]:
submission.columns

Index(['0', '1', '2', '3', '4', '5', 'id', 'price'], dtype='object')

In [105]:
submission['price'] = submission['0']

In [119]:
for i in range(len(submission)):
    # price is the index of the max value
    submission['price'][i] = submission.iloc[i, 0:-2].idxmax()


In [120]:
submission.head()

,0,1,2,3,4,5,id,price
0,0.026642,0.193753,0.256311,0.447479,0.067691,0.008125,0,3
1,0.019580,0.278200,0.430114,0.223611,0.046119,0.002377,1,2
2,0.009258,0.417058,0.421079,0.136868,0.014650,0.001086,2,2
3,0.002768,0.010895,0.089062,0.155363,0.690996,0.050916,3,4
4,0.008879,0.331591,0.207936,0.424389,0.026514,0.000691,4,3


In [122]:
submission.drop(['0', '1', '2', '3', '4', '5'], axis=1, inplace=True)

In [123]:
submission.to_csv('submission.csv', index=False)